In [1]:
import requests as r
import datetime as dt
import csv

url = 'https://api.covid19api.com/dayone/country/brazil'

response = r.get(url)

if response.status_code != 200:
	print('Nao foi possivel acessar a origem dos dados')

raw_data = response.json()

final_data = []
# header
final_data.insert(0, ['confirmados', 'obitos', 'recuperados', 'ativos', 'data'])
for obs in raw_data:
	final_data.append([obs['Confirmed'], obs['Deaths'], obs['Recovered'], obs['Active'], obs['Date']])

# constantes para auxiliar a acessar posicoes da lista
CONFIRMADOS = 0
OBITOS = 1
RECUPERADOS = 2
ATIVOS = 3
DATA = 4

for i in range(1, len(final_data)):
	final_data[i][DATA] = final_data[i][DATA][:10]

with open('brasil_covid.csv', 'w') as file:
	writer = csv.writer(file)
	writer.writerows(final_data)